# Hugging Face Example

This example demonstrates how to load models from Hugging Face and run the inference model in your program.

**Make sure to terminate your notebook kernel after you're done!**

A couple of steps are requires prior to running this exmaple:

1. You need to use a computer with an accelerator chip like NVidia GPU, Apple Silicon, etc.
2. You need to create an account on Huggin Face https://huggingface.co/ and create an Access Token https://huggingface.co/settings/tokens.
3. Some models may require you to apply for acccess and acknowledge end-user agreements. Most likely you see error messages when you try to download a particular model. Those error message include instructions and links to follow up. Approval of accessing models usually happens within a few hours.
4. This examples uses PyTorch, make sure to install the version that supports your hardware.
5. Additional packages that you need to install:
```
    transformers
    huggingface_hub
    ipywidgets
    accelerate>=0.26.0
```


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

## Check Accelerator Hardware
Check which devices are available.

In [4]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Using Apple Metal")
elif torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    devices = [torch.cuda.get_device_name(i) for i in range(device_count)]
    print(f"Available GPUs: {devices}")
    device = devices[-1]
    print(f"Using CUDA: {device}")
else:
    torch.device("cpu")
    print(f"Using CPU")

Available GPUs: ['NVIDIA L40']
Using CUDA: NVIDIA L40


Before downloading a model, you have to provide an access key to Huggingface.
Go to ... and create a key. Copy the key into a textfile that your application can load.

In [5]:
from huggingface_hub import login
token = open(".hugging_face_token.txt", "r").read().strip()
login(token)

## Where to store the downloaded model parameter
Set the path where you want to keep the model parameter. These files take up a couple of Gigabytes.

In [6]:
! mkdir -p /staging/huggin_face_cache
CACHE_DIR = "/staging/huggin_face_cache"

## Load Model

Download model parameter from Huggin Face might take a while (could be 30 minutes to an hour). Once it's on your file system this goes pretty fast.

Look out for error messages: they include links to request access if needed.

In [7]:
# model_name = "meta-llama/Llama-3.2-3B-Instruct"  # Replace with the desired model name
model_name = "meta-llama/Llama-2-7b-chat-hf"  # Replace with the desired model name"  # Replace with the desired model name
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=CACHE_DIR)
model = AutoModelForCausalLM.from_pretrained(
    model_name, cache_dir=CACHE_DIR, torch_dtype=torch.float16,
    device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Tokenize Input Text

In the first step you convert your input text into tokens. 

In [12]:
input_text = "You are What is the capital of France?"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

In [13]:
print(inputs)

{'input_ids': tensor([[    1,   887,   526,  1724,   338,   278,  7483,   310,  3444, 29973]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


## Generate Text

Then use the model to generate text. There are several attributes to control the LLM inference: https://huggingface.co/docs/transformers/en/main_classes/text_generation

In [15]:
outputs = model.generate(**inputs, max_new_tokens=100,
    return_dict_in_generate=False,
    output_scores=False
)
print(outputs)

tensor([[    1,   887,   526,  1724,   338,   278,  7483,   310,  3444, 29973,
            13,    13,  3492,   526,  1959, 29991,   450,  7483,   310,  3444,
           338,  3681, 29889,  5674,  2309, 29991,    13,    13, 12024, 29915,
         29879,  1018,  1790,  1139, 29901,    13,    13,  5618,   338,   278,
         10150, 15754,   297,  1749, 21635,  1788, 29973,     2]],
       device='cuda:0')


**Note:** If you tell the model to also return Logits and other values, your output might have a different format. You need to adjust your code accordingly.

## Decode and print the output
The model output is a sequence of tokens. You need to use the Tokenizer to decode them to readable text.

In [16]:
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

You are What is the capital of France?

You are correct! The capital of France is Paris. Well done!

Let's try another question:

What is the largest planet in our solar system?


## Create Helper Functions

You may create some functions for your own use. Like the one below:

In [27]:
import datetime
T_now = datetime.datetime.now

def llm(input_text: str, max_new_tokens: int = 1000) -> str:
    T_0 = T_now()
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    num_input = len(inputs.input_ids[0])
    print(f"Number of input tokens: {num_input:,}")
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens,
        return_dict_in_generate=False,
        output_scores=False
    )
    num_output = len(outputs[0])
    print(f"Number of output tokens: {num_output:}")
    response = tokenizer.decode(outputs[0][num_input:num_output+1], skip_special_tokens=True)
    print(f"Elapsed time: {T_now()-T_0}")
    return response
    

In [64]:
txt = """
Make a list of 17 tourist attractions in Paris, order them by their popularity with the most popular first.
For each attaction state the Name, Part of the city, short description, and number of annual visitors.
Format the output as PSV with fields "Name" | "Location" | "Description" | "Visitors".
"""

data = llm(txt, 10000)
print(data[:2000]) ### limit output ... sometime there's clutter at the end...

Number of input tokens: 79
Number of output tokens: 687
Elapsed time: 0:00:14.078426

1. Eiffel Tower | 7th | The most iconic landmark in Paris, offers panoramic views of the city | 7 million.
2. Louvre Museum | 1st | Home to Mona Lisa and thousands of other works of art | 8.5 million.
3. Notre Dame Cathedral | 4th | One of the most beautiful cathedrals in the world, famous for its Gothic architecture | 1 million.
4. Arc de Triomphe | 8th | Tall monument honoring the soldiers who fought and died for France | 7 million.
5. Champs-Elysées | 8th | Famous avenue lined with cafes, shops, and theaters | 3 million.
6. Montmartre | 18th | Historic neighborhood known for its bohemian vibe, street artists, and stunning views | 2 million.
7. Musée d'Orsay | 7th | Houses an impressive collection of Impressionist and Post-Impressionist art | 3 million.
8. Sainte-Chapelle | 1st | Known for its stunning stained glass windows and Gothic architecture | 1 million.
9. Palais Garnier | 9th | Famous opera 

## Process LLM output to structured data

You can use LLMs to produce structured data like this table of popular attractions in Paris. However, LLMs may not always produce perfectly formatted output text. Some text processing to cleanup the output might be required.

In [59]:
import pandas as pd
from io import StringIO

Let's split the output text into lines, and then split each line by the "|" (pipe symbol) 

In [68]:
raw_dat = [ list(map(lambda s: str(s).strip(), line.split('|'))) for line in data.split('\n') ]
print(raw_dat[:20])

[[''], ['1. Eiffel Tower', '7th', 'The most iconic landmark in Paris, offers panoramic views of the city', '7 million.'], ['2. Louvre Museum', '1st', 'Home to Mona Lisa and thousands of other works of art', '8.5 million.'], ['3. Notre Dame Cathedral', '4th', 'One of the most beautiful cathedrals in the world, famous for its Gothic architecture', '1 million.'], ['4. Arc de Triomphe', '8th', 'Tall monument honoring the soldiers who fought and died for France', '7 million.'], ['5. Champs-Elysées', '8th', 'Famous avenue lined with cafes, shops, and theaters', '3 million.'], ['6. Montmartre', '18th', 'Historic neighborhood known for its bohemian vibe, street artists, and stunning views', '2 million.'], ["7. Musée d'Orsay", '7th', 'Houses an impressive collection of Impressionist and Post-Impressionist art', '3 million.'], ['8. Sainte-Chapelle', '1st', 'Known for its stunning stained glass windows and Gothic architecture', '1 million.'], ['9. Palais Garnier', '9th', 'Famous opera house with 

As you can see there are some blank lines. We can filter those out and then use Pandas to convert the text into a DataFrame

In [69]:
# raw_dat = [ list(map(lambda s: str(s).strip(), line.split('|'))) for line in data.split('\n') ]
filt_dat = list(filter(lambda lst: len(lst)==4, raw_dat))
filt_dat[:5]

[['1. Eiffel Tower',
  '7th',
  'The most iconic landmark in Paris, offers panoramic views of the city',
  '7 million.'],
 ['2. Louvre Museum',
  '1st',
  'Home to Mona Lisa and thousands of other works of art',
  '8.5 million.'],
 ['3. Notre Dame Cathedral',
  '4th',
  'One of the most beautiful cathedrals in the world, famous for its Gothic architecture',
  '1 million.'],
 ['4. Arc de Triomphe',
  '8th',
  'Tall monument honoring the soldiers who fought and died for France',
  '7 million.'],
 ['5. Champs-Elysées',
  '8th',
  'Famous avenue lined with cafes, shops, and theaters',
  '3 million.']]

In [72]:
df = pd.DataFrame(filt_dat)
df.columns = ["Name", "Location", "Description", "Visitors"]
print(f"Number of rows: {df.shape[0]:,}")
display(df)

Number of rows: 17


,Name,Location,Description,Visitors
0,1. Eiffel Tower,7th,"The most iconic landmark in Paris, offers pano...",7 million.
1,2. Louvre Museum,1st,Home to Mona Lisa and thousands of other works...,8.5 million.
2,3. Notre Dame Cathedral,4th,One of the most beautiful cathedrals in the wo...,1 million.
3,4. Arc de Triomphe,8th,Tall monument honoring the soldiers who fought...,7 million.
4,5. Champs-Elysées,8th,"Famous avenue lined with cafes, shops, and the...",3 million.
5,6. Montmartre,18th,Historic neighborhood known for its bohemian v...,2 million.
6,7. Musée d'Orsay,7th,Houses an impressive collection of Impressioni...,3 million.
7,8. Sainte-Chapelle,1st,Known for its stunning stained glass windows a...,1 million.
8,9. Palais Garnier,9th,Famous opera house with opulent architecture a...,1.5 million.
9,10. Musée Grévin,8th,Wax museum featuring lifelike replicas of famo...,1.5 million.


## Travel guide function

In [83]:
def travel_guide(city: str, num_attractions: int = 17) -> pd.DataFrame:
    txt = f"""
Make a list of {num_attractions} tourist attractions in {city}, order them by their popularity with the most popular first.
For each attaction state the Name, Part of the city, short description, and number of annual visitors.
Format the output as PSV with fields "Name" | "Location" | "Description" | "Visitors".
"""
    data = llm(txt, 10000)
    raw_dat = [ list(map(lambda s: str(s).strip(), line.split('|'))) for line in data.split('\n') ]
    filt_dat = list(filter(lambda lst: len(lst)==4, raw_dat))
    df = pd.DataFrame(filt_dat, columns=["Name", "Location", "Description", "Visitors"])
    print(f"Number of rows: {df.shape[0]:,}")
    return df

In [84]:
berlin_df = travel_guide("Berlin", 5)
display(berlin_df)

Number of input tokens: 78
Number of output tokens: 288
Elapsed time: 0:00:04.828133
Number of rows: 7


,Name,Location,Description,Visitors
0,Name,Location,Description,Visitors
1,B Brandenburg Gate,Mitte,"Iconic landmark, symbol of Berlin","3,500,000"
2,1. Brandenburg Gate,Mitte,"Iconic landmark, symbol of Berlin","3,500,000"
3,2. Berlin Wall Memorial,Prenzlauer Berg,Memorial to the division of Berlin,"1,500,000"
4,3. Museum Island,Spandau,"UNESCO World Heritage Site, home to 5 museums","1,200,000"
5,4. Berlin Cathedral,Mitte,"Largest church in Berlin, impressive architecture","800,000"
6,5. Checkpoint Charlie Museum,Mitte,Museum dedicated to the history of the Berlin ...,"600,000"


In [85]:
barcelona_df = travel_guide("Barcelona", 5)
display(barcelona_df)

Number of input tokens: 78
Number of output tokens: 329
Elapsed time: 0:00:05.742739
Number of rows: 5


,Name,Location,Description,Visitors
0,1. La Sagrada Familia,Eixample,"Gaudi's masterpiece, a massive cathedral with ...","10,000,000"
1,2. Park Güell,Gràcia,"A public park designed by Gaudí, featuring stu...","8,000,000"
2,3. La Rambla,Ciutat Vella,A bustling pedestrian street lined with street...,"6,000,000"
3,4. Barceloneta Beach,Sant Martí,A popular beach with clear waters and a lively...,"5,000,000"
4,5. Casa Batlló,Eixample,"A unique and colorful house designed by Gaudí,...","4,000,000"


In [86]:
atlanta_df = travel_guide("Atlanta", 5)
display(atlanta_df)

Number of input tokens: 78
Number of output tokens: 313
Elapsed time: 0:00:05.374180
Number of rows: 5


,Name,Location,Description,Visitors
0,1. Georgia Aquarium,Downtown,Largest aquarium in the world with thousands o...,"10,000,000."
1,2. Centennial Olympic Park,Downtown,21-acre park built for the 1996 Summer Olympics,"10,000,000."
2,3. World of Coca-Cola,Downtown,Interactive museum showcasing the history of t...,"7,000,000."
3,4. Stone Mountain Park,Stone Mountain,Historic plantation and mountain with hiking t...,"6,000,000."
4,5. Martin Luther King Jr. National Historical ...,Southeast,"Site of Dr. King's birthplace, the Ebenezer Ba...","3,000,000."


### Terminate the notebook kernel after you're done to release the GPU resources!